# Eat Safe, Love

## Notebook Set Up

In [ ]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
print(db.list_collection_names())

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': {'$eq': 20}}

# Use count_documents to display the number of documents in the result
count = db.establishments.count_documents(query)
print("Number of documents with hygiene score 20:", count)
# Display the first document in the results using pprint
results = establishments.find(query)
pprint(results[0])

In [ ]:
# Convert the result to a Pandas DataFrame
hygiene_df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
length = len(hygiene_df)
print("The number of rows in Hygiene_df are:", length)
# Display the first 10 rows of the DataFrame
hygiene_df.head(10)

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
from pprint import pprint
query2 = (
    {'$and': [
        {'LocalAuthorityName': {'$regex':'London'}}, 
        {'RatingValue': {'$gte': 4}}
        ]})

# Use count_documents to display the number of documents in the result
count = db.establishments.count_documents(query2)
print("Number of documents with London as the Local Authority and has a RatingValue greater than or equal to 4:", count)
# Display the first document in the results using pprint
results2 = establishments.find(query2)
pprint(results2[0])

In [ ]:
# Convert the result to a Pandas DataFrame
london4_df = pd.DataFrame(results2)
# Display the number of rows in the DataFrame
length = len(london4_df)
print("The number of rows in Hygiene_df are:", length)
# Display the first 10 rows of the DataFrame
london4_df.head(10)

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
import pprint
degree_search = 0.01
latitude = 51.490142
longitude = 0.08384

query = {
    'RatingValue': {'$eq': 5},
    'geocode.latitude': {'$gte': latitude - degree_search, '$lte': latitude + degree_search},
    'geocode.longitude': {'$gte': longitude - degree_search, '$lte': longitude + degree_search},
}

# Sort by hygiene score in descending order
sort = [('scores.Hygiene', -1)]

# Limit the number of results
limit = 5

# Print the results
for item in establishments.find(query).sort(sort).limit(limit):
    pprint.pprint(item)

In [ ]:
# Convert result to Pandas DataFrame
results = list(establishments.find(query).sort(sort).limit(limit))

# Convert the results to a Pandas DataFrame
penang5 = pd.DataFrame(results)
penang5.head()

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
match = {'$match': {'scores.Hygiene': 0}}
group = {'$group': {'_id': {'LocalAuthorityName': "$LocalAuthorityName"}, 'count': {'$sum': 1}}}
sort = {'$sort': {'count': -1}}
pipeline = [match, group, sort]
results = list(establishments.aggregate(pipeline))
# Print the number of documents in the result
print("Number of rows in result: ", len(results))
# Print the first 10 results
print("Top 10 Local Authority establishments with hygiene score 0:")
pprint.pprint(results[:10])

In [ ]:
# Convert the result to a Pandas DataFrame
bad_hygiene_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print("Number of rows in the DataFrame:", len(bad_hygiene_df))

# Display the first 10 rows of the DataFrame
bad_hygiene_df.head(10)